In [1]:
import os
from glob import glob
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

sys.path.insert(0, '../')
from dataset import get_dataloader, LabelEncoder
from utils import save_json

In [4]:
asdf = 'qwerqwerqwer'

In [3]:
from varname import nameof

In [6]:
nameof(asdf), asdf

('asdf', 'qwerqwerqwer')

In [2]:
RANDOM_SEED = 42
TRAIN_RAW = '../input/data/train/images'

In [3]:
data = pd.read_csv('../input/data/train/train.csv')
data.head(3)

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54


In [4]:
mask_states = ['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal']

def attach_img_path(x):    
    output = [os.path.join(x, class_) for class_ in mask_states]
    return output

def get_image_paths(x):
    name = os.path.basename(x)
    output = pd.DataFrame(np.array(attach_img_path(x)), columns=['image_path'])
    output['mask_detail'] = [m for m in mask_states]
    output['path'] = name
    return output

In [5]:
%%time
path_list = data['path'].tolist()
image_paths_raw = list(map(lambda x: get_image_paths(os.path.join(TRAIN_RAW, x)), path_list))
image_paths = pd.concat(image_paths_raw, ignore_index=True, axis=0)

data = data.merge(image_paths, how='left', on='path')
data.head(3)


CPU times: user 3.16 s, sys: 12 ms, total: 3.17 s
Wall time: 3.17 s


,id,gender,race,age,path,image_path,mask_detail
0,000001,female,Asian,45,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...,incorrect_mask
1,000001,female,Asian,45,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...,mask1
2,000001,female,Asian,45,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...,mask2


In [6]:
def get_mask_class(x):
    if x == 'incorrect_mask':
        class_ = 'incorrect'
    elif x == 'normal':
        class_ = 'not_wear'
    else:
        class_ = 'wear'
    return class_


data['mask'] = data['mask_detail'].apply(lambda x: get_mask_class(x))
col_order = ['id', 'gender', 'age', 'mask', 'mask_detail', 'path', 'image_path']
data = data[col_order]
data.head(3)

,id,gender,age,mask,mask_detail,path,image_path
0,000001,female,45,incorrect,incorrect_mask,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...
1,000001,female,45,wear,mask1,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...
2,000001,female,45,wear,mask2,000001_female_Asian_45,../input/data/train/images/000001_female_Asian...


In [7]:
def tag_extension(img_path):
    for ex in ['.jpg', '.png', '.jpeg']:
        path = img_path + ex
        if os.path.isfile(path):
            break
    return path
data['image_path'] = data['image_path'].apply(lambda x: tag_extension(x))

In [8]:
img_abspath_list = data['image_path'].tolist()

In [9]:

def get_class(age: str, gender: str, mask_state) -> int:
    age_group = get_ageg(age)
    info = (mask_state, gender, age_group)
    class_ = class_meta.index(info) # 클래스 추출
    return class_

masks = ['wear', 'incorrect', 'not_wear'] # 정착용 / 불완전착용 / 미착용
genders = ['male', 'female'] # 남성 / 여성
age_groups = ['young', 'middle', 'old'] # 30대 미만 / 30대 이상 60대 미만 / 60대 이상

class_meta = []
for m in masks:
    for g in genders:
        for a in age_groups:
            class_meta.append((m, g, a))


def get_age(img_path):
    return int(img_path.split('/')[-2].split('_')[-1])

def get_ageg(age):
    if age < 30:
        age_group = 'young' # 30대 미만
    elif age >= 60:
        age_group = 'old' # 60대 이상
    else:
        age_group = 'middle' # 그 외
    return age_group

def get_mask_state(img_path):
    if 'incorrect' in img_path:
        return 'incorrect'
    elif 'normal' in img_path:
        return 'not_wear'
    else:
        return 'wear'

def get_gender(img_path):
    if 'female' in img_path:
        return 'female'
    else:
        return 'male'

In [11]:
enc = LabelEncoder()
metadata = {}

for img_path in img_abspath_list:
    name = '_'.join(img_path.split('/')[-2:])
    mask_state = get_mask_state(img_path)
    gender = get_gender(img_path)
    age = get_age(img_path)
    ageg = enc.transform(get_ageg(age), task='ageg')
    class_ = get_class(age, gender, mask_state)
    
    metadata[name] = {'mask': enc.transform(mask_state, 'mask'), 'gender': enc.transform(gender, 'gender'), 'age': age, 'ageg': ageg, 'main': class_}

In [13]:
save_json('metadata.json', metadata)